In [1]:
import pandas as pd

In [4]:
data = {
    'taxon_name': ['Pholcus phalangioides', 'Argiope keyserlingi', 'Argiope argentata', 'Sparassidae', 'Theraphosidae', 'Missulena Occatoria', 'Lampona', 'Latrodectus hasselti', 'Atrax robustus'],
    'taxon_id': [67328, 202557, 67706, 47867, 47424, 464824, 333918, 146765, 205276],
    'taxon_rank': ['species', 'species', 'species', 'family', 'family', 'species', 'genus', 'species', 'species'],
}

df = pd.DataFrame(data)
df

,taxon_name,taxon_id,taxon_rank
0,Pholcus phalangioides,67328,species
1,Argiope keyserlingi,202557,species
2,Argiope argentata,67706,species
3,Sparassidae,47867,family
4,Theraphosidae,47424,family
5,Missulena Occatoria,464824,species
6,Lampona,333918,genus
7,Latrodectus hasselti,146765,species
8,Atrax robustus,205276,species


In [30]:
import sqlite3
# Define the path to the database file
db_path = "../data/db/inat_open_data.sq3db"

# Create the URI connection string with read-only mode
uri = f"file:{db_path}?mode=ro"

# Attempt to connect to the database in read-only mode
try:
    conn = sqlite3.connect(uri, uri=True)
    print("Database connected successfully.")
except sqlite3.OperationalError as e:
    print(f"Error: {e}")
    print("The database does not exist or cannot be opened in read-only mode.")

# data = {
#     'taxon_name': ['Pholcus phalangioides', 'Argiope keyserlingi', 'Argiope argentata', 'Delena cancerides', 'Selenocosmia crassipes', 'Missulena Occatoria', 'Lampona cylindrata', 'Latrodectus hasselti', 'Atrax robustus'],
#     'taxon_id': [67328, 202557, 67706, 388489, 779693, 464824, 391331, 146765, 205276],
#     'taxon_rank': ['species', 'species', 'species', 'species', 'family', 'species', 'genus', 'species', 'species'],
# }

# excel_df = pd.DataFrame(data)
# excel_df = excel_df['taxon_id']


# taxon_list = ""
# for species in excel_df:
#     taxon_list = taxon_list + """
# OR (D.taxon_id = '"""+str(species)+"""' AND C.quality_grade = 'research' AND D.active = 1) """
# taxon_list

sq_query = """
SELECT * 
FROM taxa 
WHERE taxon_id = '67328' OR taxon_id = '202557' OR taxon_id = '67706' OR taxon_id = '388489' OR taxon_id = '779693' OR taxon_id = '464824' OR taxon_id = '391331' OR taxon_id = '146765' OR taxon_id = '205276'
"""

df = pd.read_sql(
    sq_query,
    conn
)
df


Database connected successfully.


,taxon_id,ancestry,rank_level,rank,name,active
0,67328,48460/1/47120/245097/47119/47118/120474/367158...,10.0,species,Pholcus phalangioides,1
1,67706,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Argiope argentata,1
2,146765,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Latrodectus hasselti,1
3,202557,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Argiope keyserlingi,1
4,205276,48460/1/47120/245097/47119/47118/84719/1467321...,10.0,species,Atrax robustus,1
5,388489,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Delena cancerides,1
6,391331,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Lampona cylindrata,1
7,464824,48460/1/47120/245097/47119/47118/84719/1467321...,10.0,species,Missulena occatoria,1
8,779693,48460/1/47120/245097/47119/47118/84719/1467321...,10.0,species,Selenocosmia crassipes,1


In [31]:
db_path = "../data/db/inat_open_data.sq3db"

# Create the URI connection string with read-only mode
uri = f"file:{db_path}?mode=ro"

# Attempt to connect to the database in read-only mode
try:
    conn = sqlite3.connect(uri, uri=True)
    print("Database connected successfully.")
except sqlite3.OperationalError as e:
    print(f"Error: {e}")
    print("The database does not exist or cannot be opened in read-only mode.")

# Read the Excel file into a DataFrame

# excel_df = pd.read_excel("../../data/json/6_extra_classes_fordownload.xlsx", sheet_name="Sheet1", header=1)
# excel_df = excel_df["ID in INaturalist"]

data = {
    'taxon_name': ['Pholcus phalangioides', 'Argiope keyserlingi', 'Argiope argentata', 'Delena cancerides', 'Selenocosmia crassipes', 'Missulena Occatoria', 'Lampona cylindrata', 'Latrodectus hasselti', 'Atrax robustus'],
    'taxon_id': [67328, 202557, 67706, 388489, 779693, 464824, 391331, 146765, 205276],
    'taxon_rank': ['species', 'species', 'species', 'species', 'family', 'species', 'genus', 'species', 'species'],
}

excel_df = pd.DataFrame(data)
excel_df = excel_df['taxon_id']

taxon_list = ""
for species in excel_df:
    taxon_list = taxon_list + """
OR (D.taxon_id = '"""+str(species)+"""' AND C.quality_grade = 'research' AND D.active = 1) """
taxon_list

sql_query = """
WITH urls AS (SELECT 'http://inaturalist-open-data.s3.amazonaws.com/photos/' AS photo) 
SELECT B.*, U.photo || A.photo_id || '/' || 'medium.' || A.extension AS photo_url, D.name AS taxon_name, D.ancestry, D.taxon_id, D.rank, A.photo_id, A.photo_uuid, A.extension 
FROM urls U 
CROSS JOIN photos A 
JOIN observers B 
ON A.observer_id = B.observer_id 
JOIN observations C 
ON A.observation_uuid = C.observation_uuid 
LEFT JOIN taxa D 
ON C.taxon_id = D.taxon_id 
WHERE (D.taxon_id = 'Placeholder') """ + taxon_list

# Execute the SQL query
speciesDf = pd.read_sql_query(
    sql_query,
    conn
)


speciesDf.to_csv("../data/csv/ausSpiders.csv", index=False)
speciesDf

Database connected successfully.


,observer_id,login,name,photo_url,taxon_name,ancestry,taxon_id,rank,photo_id,photo_uuid,extension
0,505,dloarie,Don Loarie,http://inaturalist-open-data.s3.amazonaws.com/...,Pholcus phalangioides,48460/1/47120/245097/47119/47118/120474/367158...,67328,species,23047,1814af90-6f4c-4238-98cb-36f3daa15216,jpg
1,505,dloarie,Don Loarie,http://inaturalist-open-data.s3.amazonaws.com/...,Pholcus phalangioides,48460/1/47120/245097/47119/47118/120474/367158...,67328,species,23048,f9dac87c-5fc9-444e-b67b-6ba20767d056,jpg
2,169,flapack,None,http://inaturalist-open-data.s3.amazonaws.com/...,Argiope argentata,48460/1/47120/245097/47119/47118/120474/342614...,67706,species,25153,8038f1a8-fc8a-42bc-beb0-8b820758d943,jpg
3,169,flapack,None,http://inaturalist-open-data.s3.amazonaws.com/...,Argiope argentata,48460/1/47120/245097/47119/47118/120474/342614...,67706,species,25154,c8a3f219-cab0-4ea2-86e8-577100b85484,jpg
4,1867,leopimentaz,Léo Pimenta,http://inaturalist-open-data.s3.amazonaws.com/...,Argiope argentata,48460/1/47120/245097/47119/47118/120474/342614...,67706,species,38219,44b72cdd-b818-41e7-b3a5-6cb0e4a46876,jpg
...,...,...,...,...,...,...,...,...,...,...,...
61435,6290629,zova,None,http://inaturalist-open-data.s3.amazonaws.com/...,Pholcus phalangioides,48460/1/47120/245097/47119/47118/120474/367158...,67328,species,371456233,276fc10a-75b7-4db7-8b04-ccfaf9cfe0d4,jpeg
61436,6290629,zova,None,http://inaturalist-open-data.s3.amazonaws.com/...,Pholcus phalangioides,48460/1/47120/245097/47119/47118/120474/367158...,67328,species,371456293,26a682b3-9e81-4640-9c58-8fdce68ebc8f,jpeg
61437,6290629,zova,None,http://inaturalist-open-data.s3.amazonaws.com/...,Pholcus phalangioides,48460/1/47120/245097/47119/47118/120474/367158...,67328,species,371456354,57685e5c-63c0-444f-9c5b-42b842f570d3,jpeg
61438,6290629,zova,None,http://inaturalist-open-data.s3.amazonaws.com/...,Pholcus phalangioides,48460/1/47120/245097/47119/47118/120474/367158...,67328,species,371456409,6bcf8e6a-52c7-45de-9cc5-f7dc0c5a8b18,jpeg


In [ ]:
db_path = "../data/db/inat_open_data.sq3db"

# Create the URI connection string with read-only mode
uri = f"file:{db_path}?mode=ro"

# Attempt to connect to the database in read-only mode
try:
    conn = sqlite3.connect(uri, uri=True)
    print("Database connected successfully.")
except sqlite3.OperationalError as e:
    print(f"Error: {e}")
    print("The database does not exist or cannot be opened in read-only mode.")

# Read the Excel file into a DataFrame

# excel_df = pd.read_excel("../../data/json/6_extra_classes_fordownload.xlsx", sheet_name="Sheet1", header=1)
# excel_df = excel_df["ID in INaturalist"]

data = {
    'taxon_name': ['Pholcus phalangioides', 'Argiope keyserlingi', 'Argiope argentata', 'Delena cancerides', 'Selenocosmia crassipes', 'Missulena Occatoria', 'Lampona cylindrata', 'Latrodectus hasselti', 'Atrax robustus'],
    'taxon_id': [67328, 202557, 67706, 388489, 779693, 464824, 391331, 146765, 205276],
    'taxon_rank': ['species', 'species', 'species', 'species', 'family', 'species', 'genus', 'species', 'species'],
}

excel_df = pd.DataFrame(data)
excel_df = excel_df['taxon_id']


taxon_list = ""
for species in excel_df:
    taxon_list = taxon_list + """
OR (D.taxon_id = '"""+str(species)+"""' AND C.quality_grade = 'research' AND D.active = 1) """
taxon_list

# taxon_list = ""
# for species in excel_df:
#     taxon_list = taxon_list + """
# OR (D.ancestry LIKE '%"""+str(species)+"""%' AND C.quality_grade = 'research' AND D.active = 1 AND D.rank_level = '10.0') """
# taxon_list

# Define the SQL query
sql_query = """
WITH urls AS (SELECT 'http://inaturalist-open-data.s3.amazonaws.com/photos/' AS photo) 
SELECT B.*, U.photo || A.photo_id || '/' || 'medium.' || A.extension AS photo_url, D.name AS taxon_name, D.ancestry, D.taxon_id, D.rank, A.photo_id, A.photo_uuid, A.extension 
FROM urls U 
CROSS JOIN photos A 
JOIN observers B 
ON A.observer_id = B.observer_id 
JOIN observations C 
ON A.observation_uuid = C.observation_uuid 
LEFT JOIN taxa D 
ON C.taxon_id = D.taxon_id 
WHERE (D.taxon_id = 'Placeholder') """ + taxon_list

# Execute the SQL query
speciesDf = pd.read_sql_query(
    sql_query,
    conn
)


speciesDf

In [19]:
# Define the SQL query
sql_query = """
WITH urls AS (SELECT 'http://inaturalist-open-data.s3.amazonaws.com/photos/' AS photo) 
SELECT B.*, U.photo || A.photo_id || '/' || 'medium.' || A.extension AS photo_url, D.name AS taxon_name, D.ancestry, D.taxon_id, D.rank, A.photo_id, A.photo_uuid, A.extension 
FROM urls U 
CROSS JOIN photos A 
JOIN observers B 
ON A.observer_id = B.observer_id 
JOIN observations C 
ON A.observation_uuid = C.observation_uuid 
LEFT JOIN taxa D 
ON C.taxon_id = D.taxon_id 
WHERE (D.taxon_id = 'Placeholder') """ + taxon_list

# Execute the SQL query
urlDf = pd.read_sql_query(
    sql_query,
    conn
)

urlDf.to_csv("new_urls.csv", index=False)

urlDf



,observer_id,login,name,photo_url,taxon_name,ancestry,taxon_id,rank,photo_id,photo_uuid,extension
0,1000,muir,Matt Muir,http://inaturalist-open-data.s3.amazonaws.com/...,Avicularia rufa,48460/1/47120/245097/47119/47118/84719/1467321...,567360,species,47681,16d66884-1d90-44c1-a91e-7b73ff318b09,jpg
1,1000,muir,Matt Muir,http://inaturalist-open-data.s3.amazonaws.com/...,Avicularia rufa,48460/1/47120/245097/47119/47118/84719/1467321...,567360,species,47682,6653e4c6-53a0-49d5-85e8-b3ed2fde5fcc,jpg
2,710,johngcramer,John Cramer III,http://inaturalist-open-data.s3.amazonaws.com/...,Aphonopelma johnnycashi,48460/1/47120/245097/47119/47118/84719/1467321...,501065,species,48870,e0aef409-4cc1-4477-a259-1328f736c2d9,jpg
3,1,kueda,Ken-ichi Ueda,http://inaturalist-open-data.s3.amazonaws.com/...,Aphonopelma iodius,48460/1/47120/245097/47119/47118/84719/1467321...,180759,species,57035,43d19e6b-c06e-4136-90ed-8665d9aaf7b7,jpg
4,1,kueda,Ken-ichi Ueda,http://inaturalist-open-data.s3.amazonaws.com/...,Aphonopelma iodius,48460/1/47120/245097/47119/47118/84719/1467321...,180759,species,57036,f85fbbcc-1bc1-481d-ab0c-fe286ad78264,jpg
...,...,...,...,...,...,...,...,...,...,...,...
84801,1684158,brookewilder523,None,http://inaturalist-open-data.s3.amazonaws.com/...,Heteropoda venatoria,48460/1/47120/245097/47119/47118/120474/342614...,48765,species,371437006,a86ff4f6-e792-4193-b7f2-7063401c3e41,jpg
84802,1979727,terryleecrane,Terry Lee Crane,http://inaturalist-open-data.s3.amazonaws.com/...,Aphonopelma anax,48460/1/47120/245097/47119/47118/84719/1467321...,123097,species,371475529,af236c1f-6448-411c-bd15-e94763a69547,jpeg
84803,1979727,terryleecrane,Terry Lee Crane,http://inaturalist-open-data.s3.amazonaws.com/...,Aphonopelma anax,48460/1/47120/245097/47119/47118/84719/1467321...,123097,species,371456681,39454061-afd5-4b87-842b-65cb036ff0c1,jpeg
84804,2859524,gecko9,Terry,http://inaturalist-open-data.s3.amazonaws.com/...,Isopedella victorialis,48460/1/47120/245097/47119/47118/120474/342614...,484933,species,371491958,137601be-c604-413e-a97c-45027511bd31,jpeg


In [32]:
countDf = speciesDf.groupby('taxon_name').size().reset_index(name='count').sort_values(by='count', ascending=False)
topSpecies = countDf[0:20]
topSpecies


,taxon_name,count
0,Argiope argentata,30443
7,Pholcus phalangioides,17088
1,Argiope keyserlingi,6709
5,Latrodectus hasselti,3867
3,Delena cancerides,2040
4,Lampona cylindrata,532
6,Missulena occatoria,488
2,Atrax robustus,242
8,Selenocosmia crassipes,31


In [5]:
import pandas as pd

In [26]:
df = pd.read_excel("../data/Table S1.xlsx", sheet_name=1, header=5)

In [27]:
df['Unnamed: 23']

0                      Agelena limbata; Agelena tungchis
1                                    Allagelena opulenta
2                          Anahita fauna; Anahita maolan
3      Anelosimus chonganicus; Anelosimus crassipes; ...
4      Anyphaena pugil; Anyphaena taiwanensis; Anypha...
                             ...                        
115    Tylorida mengla; Tylorida striata; Tylorida ti...
116                                  Typopeltis crucifer
117    Uloborus formosanus; Uloborus penicillatoides;...
118    Xysticus chui; Xysticus croceus; Xysticus ephi...
119                                     Zosis geniculata
Name: Unnamed: 23, Length: 120, dtype: object

In [28]:
all_species = []
for i in range(0, len(df)):
    split = df['Unnamed: 23'][i].split("; ")
    #print(split)
    for species in split:
        all_species.append(species)

print(all_species)

['Agelena limbata', 'Agelena tungchis', 'Allagelena opulenta', 'Anahita fauna', 'Anahita maolan', 'Anelosimus chonganicus', 'Anelosimus crassipes', 'Anelosimus taiwanicus', 'Anyphaena pugil', 'Anyphaena taiwanensis', 'Anyphaena wuyi', 'Arachnura logio', 'Arachnura melanura', 'Arachnura spinosa', 'Araneus seminiger', 'Araneus viridiventris', 'Araneus ejusmodi', 'Araneus inustus', 'Araniella yaginumai', 'Arctosa indica', 'Arctosa labiata', 'Arctosa laminata', 'Arctosa truncata', 'Argiope aemula', 'Argiope aetheroides', 'Argiope amoena', 'Argiope chloreis', 'Argiope minuta', 'Argiope ocula', 'Argiope perforata', 'Argyrodes bonadea', 'Argyrodes fissifrons', 'Argyrodes flavescens', 'Argyrodes kumadai', 'Argyrodes lanyuensis', 'Argyrodes miniaceus', 'Ariamnes cylindrogaster', 'Asemonea tanikawai', 'Atypus formosensis', 'Atypus karschi', 'Bavia aericeps', 'Bijoaraneus mitificus', 'Bothriocyrtum tractabile', 'Carrhotus sannio', 'Carrhotus tristis', 'Carrhotus viduus', 'Carrhotus xanthogramma',

In [29]:
# Use pandas' `str.split()` method to split the values in the column and `explode()` to flatten the list
all_species = df['Unnamed: 23'].str.split("; ").explode().tolist()

print(len(all_species))


321


In [31]:
import sqlite3
# Define the path to the database file
#db_path = "../data/db/inat_open_data.sq3db"
db_path = "../../../LDAWT/LDAWT-iNaturalist/data/inat_open_data.sq3db"
# Create the URI connection string with read-only mode
uri = f"file:{db_path}?mode=ro"

# Attempt to connect to the database in read-only mode
try:
    conn = sqlite3.connect(uri, uri=True)
    print("Database connected successfully.")
except sqlite3.OperationalError as e:
    print(f"Error: {e}")
    print("The database does not exist or cannot be opened in read-only mode.")

taxon_id_string = ""
for species in all_species:
    taxon_id_string = taxon_id_string + """
OR (name = '"""+species+"""') """

sq_query = """
SELECT * 
FROM taxa 
WHERE name = 'Placeholder' """ + taxon_id_string

taxa_df = pd.read_sql(
    sq_query,
    conn
)
taxa_df

Database connected successfully.


,taxon_id,ancestry,rank_level,rank,name,active
0,499227,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Agelena limbata,1
1,741276,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Agelena tungchis,1
2,360521,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Allagelena opulenta,1
3,360547,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Anahita fauna,1
4,741286,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Anahita maolan,1
...,...,...,...,...,...,...
289,741451,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Xysticus chui,1
290,357994,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Xysticus croceus,1
291,360594,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Xysticus ephippiatus,1
292,360590,48460/1/47120/245097/47119/47118/120474/342614...,10.0,species,Xysticus kurilensis,1


In [33]:
db_path = "../../../LDAWT/LDAWT-iNaturalist/data/inat_open_data.sq3db"

# Create the URI connection string with read-only mode
uri = f"file:{db_path}?mode=ro"

# Attempt to connect to the database in read-only mode
try:
    conn = sqlite3.connect(uri, uri=True)
    print("Database connected successfully.")
except sqlite3.OperationalError as e:
    print(f"Error: {e}")
    print("The database does not exist or cannot be opened in read-only mode.")

taxon_list = ""
for species in taxa_df['taxon_id']:
    taxon_list = taxon_list + """
OR (D.taxon_id = '"""+str(species)+"""' AND C.quality_grade = 'research' AND D.active = 1) """
print(taxon_list)

sql_query = """
WITH urls AS (SELECT 'http://inaturalist-open-data.s3.amazonaws.com/photos/' AS photo) 
SELECT B.*, U.photo || A.photo_id || '/' || 'medium.' || A.extension AS photo_url, D.name AS taxon_name, D.ancestry, D.taxon_id, D.rank, A.photo_id, A.photo_uuid, A.extension 
FROM urls U 
CROSS JOIN photos A 
JOIN observers B 
ON A.observer_id = B.observer_id 
JOIN observations C 
ON A.observation_uuid = C.observation_uuid 
LEFT JOIN taxa D 
ON C.taxon_id = D.taxon_id 
WHERE (D.taxon_id = 'Placeholder') """ + taxon_list

# Execute the SQL query
speciesDf = pd.read_sql_query(
    sql_query,
    conn
)


speciesDf.to_csv("../data/csv/taiwanSpiders.csv", index=False)
speciesDf

Database connected successfully.

OR (D.taxon_id = '499227' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '741276' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '360521' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '360547' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '741286' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '740823' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '360386' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '741285' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '360407' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '819881' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '741283' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '360472' AND C.quality_grade = 'research' AND D.active = 1) 
OR (D.taxon_id = '715179' AND 

OSError: Cannot save file into a non-existent directory: '../data/csv'

In [35]:
speciesDf.to_csv("taiwanSpiders.csv", index=False)